In [1]:
import sys
sys.path.append('/Users/jiechen/Apoex/Email/fastAPI')
from app.workflow.categorize_connect import CategoryConnect
from app.services.utils import fetchFromDB
from app.services.base_service import BaseService
import pandas as pd

bs = BaseService()
# Check if any reliable matches were found during connection
# id = 194949
# email = fetchFromDB(bs.email_spec_query.format(EMAILID=id))
email = pd.read_json(f'data/test_data/email8.json')
email.to_json(f'data/test_data/cate{id}.json', orient="records", indent=4, force_ascii=False)
# email = pd.read_json(f'data/test_data/email8.json')
cc = CategoryConnect(df=email)
ca = cc.do_connect()
ca

,id,from,sender,source,to,receiver,category,errandId,totalAmount,settlementAmount,reference,insuranceCaseRef,insuranceNumber,damageNumber,animalName,ownerName,note,connectedCol,showPage,isStaffAnimal
0,189142,djurskador@folksam.se,Folksam,Insurance_Company,klinik-djurhalsan@direktregleringsportalen.se,Djurhälsan i Dalarna,Settlement_Approved,[72686],4805.0,2254.0,1000723682,1000723682,FF093480101S,FF093480101S,Murre,PIA LENA ELFGREN,Reliable,reference,Ersätt,False
1,189145,Dir.HundKatt@dina.se,Dina Försäkringar,Insurance_Company,klinik-halsingevetsoderhamn@direktregleringspo...,Hälsinge Smådjursklinik Söderhamn,Settlement_Approved,[72545],1721.0,0.0,1000722270,1000722270,263454693,<NA>,Ruff,Anna Hanold,Reliable,reference,Koppla,False
2,189147,djurskador@folksam.se,Folksam,Insurance_Company,klinik-cityvet@direktregleringsportalen.se,Cityveterinären Malmö,Settlement_Approved,[72623],375.0,281.0,1000723054,1000723054,FF093171882S,FF093171882S,Teddi,LISELOTT SANDSTRÖM,Reliable,reference,Ersätt,False
3,189149,djurskador@if.se,If,Insurance_Company,klinik-halsingevethudik@direktregleringsportal...,Hälsinge Smådjursklinik Hudiksvall,Manual_Handling_Required,[],<NA>,<NA>,66821,<NA>,<NA>,251 744 699,Lilla Stjärnan,Larsson Jens Daniel,Unreliable,None,Koppla,False
4,189150,direktreglering@lassie.co,Lassie,Insurance_Company,klinik-djurklinikkiruna@direktregleringsportal...,Lapplands Djurklinik Kiruna,Settlement_Approved,[72691],3085.0,<NA>,1000723732,1000723732,VOFF-VALTER-220607-19,072425-44627-60*,<NA>,Ida Rova,Reliable,reference,Ersätt,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,191488,djurskador@svedea.se,Svedea,Insurance_Company,"klinik-dingle@direktregleringsportalen.se, t.m...",Dingleveterinärerna,Settlement_Approved,[73531],7943.0,5957.0,1000732139,1000732139,5843439,HU25-5843439-11954,Hedda,RETURADRESS,Reliable,reference,Ersätt,False
854,191489,erik.hadfy@apoex.se,DRP,DRP,mail@drp.se,DRP,Manual_Handling_Required,[],<NA>,<NA>,3877473579,<NA>,<NA>,<NA>,<NA>,<NA>,Unreliable,None,Koppla,False
855,191500,djurskador@svedea.se,Svedea,Insurance_Company,"ann_sofieekman@hotmail.com, klinik-djurklinikl...",Lapplands Djurklinik Luleå,Settlement_Approved,[73586],5160.0,3870.0,1000732683,1000732683,8360759,HU25-8360759-13836,Musse (Klubban),RETURADRESS,Reliable,reference,Ersätt,False
856,191502,djurskador@folksam.se,Folksam,Insurance_Company,klinik-halsingevethudik@direktregleringsportal...,Hälsinge Smådjursklinik Hudiksvall,Settlement_Approved,[73588],2342.0,1618.0,1000732709,1000732709,FF093344760S,FF093344760S,Disel,STINA HAGLUNDH,Reliable,reference,Ersätt,False


In [2]:
from app.services.classifier import Classifier
a = Classifier()
a.statistic(ca)

Debug: Note column values: {'Reliable': 622, 'Unreliable': 236}
Debug: Reliable count: 622
all_records: 858, auto_categorized: 810, 94.41%, manual: 48, 5.59%, 
                  connected_to_errands: 709, 82.63%, single_errandId: 709, 82.63%,                   reliable_connection: 622, 72.49%, staff_animal: 0, 0.00%
 - Auto_Reply: 71, connect:47, un-connect: 24
 - Finance_Report: 6, connect:0, un-connect: 6
 - Wisentic_Error: 15, connect:15, un-connect: 0
 - Other: 2, connect:0, un-connect: 2
 - Information: 0, connect:0, un-connect: 0
 - Insurance_Validation_Error: 5, connect:4, un-connect: 1
 - Settlement_Denied: 4, connect:4, un-connect: 0
 - Settlement_Approved: 506, connect:498, un-connect: 8
 - Settlement_Request: 7, connect:0, un-connect: 7
 - Message: 7, connect:6, un-connect: 1
 - Question: 8, connect:5, un-connect: 3
 - Spam: 1, connect:0, un-connect: 1
 - Complement_DR_Insurance_Company: 49, connect:49, un-connect: 0
 - Complement_Damage_Request_Insurance_Company: 56, connec

{'total_records': 858,
 'auto_categorized': '810 (94.4%)',
 'manual_handling_required': '48 (5.6%)',
 'connected_to_errands': '709 (82.6%)',
 'single_errand_connection': '709 (82.6%)',
 'reliable_connections': '622 (72.5%)',
 'staff_animals': '0 (0.0%)'}